In [6]:
# Two-Stage Demand Forecasting (Favorita) — Occurrence + Size

import os, gc, json, warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")

# ---- Paths ----
RAW = "data/raw"
PROCESSED = "data/processed"
MODELS = "models"
REPORTS = "reports"
os.makedirs(PROCESSED, exist_ok=True)
os.makedirs(MODELS, exist_ok=True)
os.makedirs(REPORTS, exist_ok=True)

# ---- Config ----
HORIZON_DAYS = 14
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

SPEED_MODE = True
DATE_START = "2017-01-01"
MAX_STORES = 10
MAX_ITEMS  = 200

import sys
print("Python:", sys.executable)


Python: C:\Users\mnsuh\OneDrive\Desktop\darkstore\venv\Scripts\python.exe


In [7]:

# Peek at the header to choose the correct columns
hdr = pd.read_csv(f"{RAW}/train.csv", nrows=3)

if "item_nbr" in hdr.columns:
    # Older 2017 Favorita
    USECOLS = ["date", "store_nbr", "item_nbr", "unit_sales", "onpromotion"]
    KEY_ITEM = "item_nbr"
    TARGET   = "unit_sales"
else:
    # Newer "Store Sales" competition (most likely your case)
    USECOLS = ["date", "store_nbr", "family", "sales", "onpromotion"]
    KEY_ITEM = "family"
    TARGET   = "sales"

train = pd.read_csv(f"{RAW}/train.csv", usecols=USECOLS, parse_dates=["date"])

# Standardize to our schema
df = train.rename(columns={
    "date": "date",
    "store_nbr": "store_id",
    KEY_ITEM: "sku_id",
    TARGET: "qty",
    "onpromotion": "is_promo"
})
df["date"] = pd.to_datetime(df["date"])
df["qty"] = df["qty"].clip(lower=0)
df["is_promo"] = df["is_promo"].fillna(False).astype(int)

# Optional: trim to a smaller slice for laptop training
if SPEED_MODE:
    df = df[df["date"] >= DATE_START].copy()
    top_stores = (df.groupby("store_id")["qty"].sum()
                    .sort_values(ascending=False).head(MAX_STORES).index)
    df = df[df["store_id"].isin(top_stores)]
    top_items = (df.groupby("sku_id")["qty"].sum()
                   .sort_values(ascending=False).head(MAX_ITEMS).index)
    df = df[df["sku_id"].isin(top_items)]

# Categories save memory and help LightGBM
for c in ["store_id","sku_id"]:
    df[c] = df[c].astype("category")

df.sort_values(["store_id","sku_id","date"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()


,date,store_id,sku_id,qty,is_promo
0,2017-01-01,3,AUTOMOTIVE,0.0,0
1,2017-01-02,3,AUTOMOTIVE,9.0,0
2,2017-01-03,3,AUTOMOTIVE,5.0,0
3,2017-01-04,3,AUTOMOTIVE,19.0,0
4,2017-01-05,3,AUTOMOTIVE,13.0,0


In [8]:
# Load side tables
stores = pd.read_csv(f"{RAW}/stores.csv")  # has store_nbr, city, state, etc.
hol    = pd.read_csv(f"{RAW}/holidays_events.csv", parse_dates=["date"])
oil    = pd.read_csv(f"{RAW}/oil.csv", parse_dates=["date"]).rename(columns={"dcoilwtico":"oil_price"})
txn    = pd.read_csv(f"{RAW}/transactions.csv", parse_dates=["date"])

# Align transactions store column name
txn.rename(columns={"store_nbr":"store_id"}, inplace=True)

# Merge transactions
if SPEED_MODE:
    txn = txn[txn["date"] >= df["date"].min()]
df = df.merge(txn, how="left", on=["store_id","date"])
df["transactions"] = df.groupby("store_id")["transactions"].transform(lambda s: s.fillna(s.median()))
df["transactions"] = df["transactions"].fillna(df["transactions"].median())

# Merge oil; fill gaps
oil = oil.sort_values("date")
oil["oil_price"] = oil["oil_price"].ffill().bfill()
df = df.merge(oil[["date","oil_price"]], how="left", on="date")
df["oil_price"] = df["oil_price"].ffill().bfill()

# Simple holiday flag: mark only real holidays (ignore Work Day / Transfer)
hol_flag = (hol[hol["type"].isin(["Holiday","Additional"])]
              .loc[~hol["transferred"].fillna(False), ["date"]]
              .drop_duplicates())
hol_flag["holiday_flag"] = 1
df = df.merge(hol_flag, on="date", how="left")
df["holiday_flag"] = df["holiday_flag"].fillna(0).astype(int)

df.head()


,date,store_id,sku_id,qty,is_promo,transactions,oil_price,holiday_flag
0,2017-01-01,3,AUTOMOTIVE,0.0,0,3088.0,53.75,0
1,2017-01-02,3,AUTOMOTIVE,9.0,0,3918.0,53.75,0
2,2017-01-03,3,AUTOMOTIVE,5.0,0,3406.0,52.36,0
3,2017-01-04,3,AUTOMOTIVE,19.0,0,3382.0,53.26,0
4,2017-01-05,3,AUTOMOTIVE,13.0,0,3109.0,53.77,0


In [9]:
def reindex_group(g):
    idx = pd.date_range(g["date"].min(), g["date"].max(), freq="D")
    g = g.set_index("date").reindex(idx)
    g.index.name = "date"
    g["store_id"] = g["store_id"].iloc[0]
    g["sku_id"]   = g["sku_id"].iloc[0]
    g["qty"] = g["qty"].fillna(0)
    g["is_promo"] = g["is_promo"].fillna(0).astype(int)
    for c in ["transactions","oil_price","holiday_flag"]:
        if c in g.columns:
            g[c] = g[c].ffill().bfill().fillna(0)
    return g.reset_index()

df = (df.groupby(["store_id","sku_id"], group_keys=False)
        .apply(reindex_group)
        .sort_values(["store_id","sku_id","date"])
        .reset_index(drop=True))
df.head(), df.shape


(        date  store_id      sku_id   qty  is_promo  transactions  oil_price  \
 0 2017-01-01         3  AUTOMOTIVE   0.0         0        3088.0      53.75   
 1 2017-01-02         3  AUTOMOTIVE   9.0         0        3918.0      53.75   
 2 2017-01-03         3  AUTOMOTIVE   5.0         0        3406.0      52.36   
 3 2017-01-04         3  AUTOMOTIVE  19.0         0        3382.0      53.26   
 4 2017-01-05         3  AUTOMOTIVE  13.0         0        3109.0      53.77   
 
    holiday_flag  
 0             0  
 1             0  
 2             0  
 3             0  
 4             0  ,
 (74910, 8))

In [10]:
def add_calendar(X):
    d = pd.to_datetime(X["date"])
    X["dow"] = d.dt.weekday
    X["month"] = d.dt.month
    X["is_weekend"] = (X["dow"]>=5).astype(int)
    X["is_month_end"] = d.dt.is_month_end.astype(int)
    return X

def add_lags_rolls(X, target="qty"):
    X = X.sort_values(["store_id","sku_id","date"]).copy()
    g = X.groupby(["store_id","sku_id"], sort=False)
    X["lag_1"]  = g[target].shift(1)
    X["lag_7"]  = g[target].shift(7)
    X["lag_14"] = g[target].shift(14)
    X["roll7_mean"]  = g[target].shift(1).rolling(7).mean()
    X["roll7_std"]   = g[target].shift(1).rolling(7).std()
    X["roll28_mean"] = g[target].shift(1).rolling(28).mean()
    X["exp_decay_7"] = g[target].shift(1).ewm(halflife=7, min_periods=3).mean()
    return X

def add_store_prior(X, target="qty"):
    X = X.sort_values(["store_id","date"]).copy()
    X["store_roll7_mean"] = X.groupby("store_id")[target].shift(1).rolling(7).mean()
    return X

feat = df.copy()
feat = add_calendar(feat)
feat = add_lags_rolls(feat, target="qty")
feat = add_store_prior(feat, target="qty")

feature_cols = [
    "dow","month","is_weekend","is_month_end",
    "lag_1","lag_7","lag_14","roll7_mean","roll7_std","roll28_mean","exp_decay_7",
    "store_roll7_mean",
    "is_promo","transactions","oil_price","holiday_flag"
]

# Clean NaNs from the early days of each series
for c in feature_cols:
    feat[c] = feat[c].replace([np.inf,-np.inf], np.nan)

def drop_cold_start(g):
    m = g["roll28_mean"].isna()
    if m.any():
        first_ok = (~m).idxmax()
        return g.loc[first_ok:]
    return g

feat = (feat.groupby(["store_id","sku_id"], group_keys=False)
            .apply(drop_cold_start)
            .reset_index(drop=True))

feat[feature_cols] = feat[feature_cols].fillna(0.0)

# Targets for two-stage
feat["occ"]  = (feat["qty"]>0).astype(int)
feat["size"] = feat["qty"].clip(lower=0).astype(float)

for c in ["store_id","sku_id"]:
    feat[c] = feat[c].astype("category")

feat.shape, feat.head()


((65670, 22),
         date store_id      sku_id   qty  is_promo  transactions  oil_price  \
 0 2017-01-29        3  AUTOMOTIVE   8.0         0        3406.0      53.18   
 1 2017-01-30        3  AUTOMOTIVE  15.0         0        2671.0      52.63   
 2 2017-01-31        3  AUTOMOTIVE   4.0         0        2938.0      52.75   
 3 2017-02-01        3  AUTOMOTIVE   6.0         0        3347.0      53.90   
 4 2017-02-02        3  AUTOMOTIVE   7.0         0        2890.0      53.55   
 
    holiday_flag  dow  month  ...  lag_1  lag_7  lag_14  roll7_mean  roll7_std  \
 0             0    6      1  ...   18.0   22.0    23.0   13.571429   6.502747   
 1             0    0      1  ...    8.0   12.0     1.0   11.571429   5.563486   
 2             0    1      1  ...   15.0    3.0     7.0   12.000000   5.715476   
 3             0    2      2  ...    4.0   10.0    17.0   12.142857   5.459810   
 4             0    3      2  ...    6.0   11.0     2.0   11.571429   5.912054   
 
    roll28_mean 

In [11]:
def rolling_cv_indices(dates: pd.Series, n_splits=4, min_train_days=150, holdout_days=HORIZON_DAYS):
    unique_days = np.array(sorted(pd.unique(pd.to_datetime(dates))))
    folds = []
    for i in range(n_splits):
        train_end = len(unique_days) - (n_splits - i) * holdout_days
        if train_end < min_train_days:
            continue
        tr_days = unique_days[:train_end]
        va_days = unique_days[train_end:train_end+holdout_days]
        folds.append((tr_days, va_days))
    for tr_days, va_days in folds:
        tr_idx = np.where(pd.to_datetime(dates).isin(tr_days))[0]
        va_idx = np.where(pd.to_datetime(dates).isin(va_days))[0]
        yield tr_idx, va_idx

def WAPE(y_true, y_pred, eps=1e-9):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps))

def SMAPE(y_true, y_pred, eps=1e-9):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.mean(2*np.abs(y_true - y_pred)/(np.abs(y_true)+np.abs(y_pred)+eps)))

def BIAS(y_true, y_pred, eps=1e-9):
    y_true, y_pred = np.asarray(y_true), np.asarray(y_pred)
    return float(np.sum(y_pred - y_true) / (np.sum(np.abs(y_true))+eps))


In [12]:
dates = feat["date"]
X = feat[feature_cols]
y_occ  = feat["occ"]
y_size = feat["size"]
y_true = feat["qty"].values

preds_oof = np.zeros(len(feat))
fold_summ = []

clf = LGBMClassifier(
    n_estimators=600, learning_rate=0.05, num_leaves=127,
    subsample=0.9, colsample_bytree=0.9, random_state=RANDOM_STATE
)
reg = LGBMRegressor(
    objective="tweedie", tweedie_variance_power=1.2,
    n_estimators=900, learning_rate=0.045, num_leaves=255,
    subsample=0.9, colsample_bytree=0.9, reg_alpha=1.0, reg_lambda=2.0,
    random_state=RANDOM_STATE
)

for tr_idx, va_idx in tqdm(list(rolling_cv_indices(dates, n_splits=4, min_train_days=150, holdout_days=HORIZON_DAYS))):
    Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
    ytr_occ, yva_occ = y_occ.iloc[tr_idx], y_occ.iloc[va_idx]
    ytr_size, yva_size = y_size.iloc[tr_idx], y_size.iloc[va_idx]
    yva_true = y_true[va_idx]

    # Stage 1: occurrence
    clf.fit(Xtr, ytr_occ)
    p_occ = clf.predict_proba(Xva)[:,1]

    # Stage 2: size (train only on positives)
    mask_pos = ytr_size > 0
    reg.fit(Xtr[mask_pos], ytr_size[mask_pos])

    size_pred = np.clip(reg.predict(Xva), 0, None)
    yhat = p_occ * size_pred

    preds_oof[va_idx] = yhat
    fold_wape = WAPE(yva_true, yhat)
    fold_smape = SMAPE(yva_true, yhat)
    fold_bias = BIAS(yva_true, yhat)
    try:
        fold_auc = roc_auc_score(yva_occ, p_occ)
    except Exception:
        fold_auc = np.nan

    fold_summ.append({"wape":fold_wape, "smape":fold_smape, "bias":fold_bias, "occ_auc":fold_auc})
    gc.collect()

cv_avg = pd.DataFrame(fold_summ).mean(numeric_only=True).to_dict()
cv_avg


  0%|                                                                                 | 0/3 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 48396, number of negative: 3414
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004372 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2580
[LightGBM] [Info] Number of data points in the train set: 51810, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.934105 -> initscore=2.651533
[LightGBM] [Info] Start training from score 2.651533
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2580
[LightGBM] [Info] Number of data points in the train set: 48396, number of used features: 16
[LightGBM] [Info] Start training from score 6.986608


 33%|████████████████████████▎                                                | 1/3 [00:10<00:21, 10.91s/it]

[LightGBM] [Info] Number of positive: 52630, number of negative: 3800
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2591
[LightGBM] [Info] Number of data points in the train set: 56430, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.932660 -> initscore=2.628285
[LightGBM] [Info] Start training from score 2.628285
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2591
[LightGBM] [Info] Number of data points in the train set: 52630, number of used features: 16
[LightGBM] [Info] Start training from score 6.983918


 67%|████████████████████████████████████████████████▋                        | 2/3 [00:19<00:09,  9.69s/it]

[LightGBM] [Info] Number of positive: 56816, number of negative: 4234
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002748 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2606
[LightGBM] [Info] Number of data points in the train set: 61050, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.930647 -> initscore=2.596671
[LightGBM] [Info] Start training from score 2.596671
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002731 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2606
[LightGBM] [Info] Number of data points in the train set: 56816, number of used features: 16
[LightGBM] [Info] Start training from score 6.983361


100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:28<00:00,  9.64s/it]


{'wape': 0.12381067320174077,
 'smape': 0.4050997645856913,
 'bias': 0.07559840381273646,
 'occ_auc': 0.9922994627145019}

In [13]:
# A/B/C by cumulative volume within each (store, sku) pair overall
vol = feat.groupby(["store_id","sku_id"])["qty"].sum().sort_values(ascending=False)
cum = (vol.cumsum()/vol.sum()).reset_index(name="cumshare")
abc = pd.merge(vol.reset_index(name="total_qty"), cum, on=["store_id","sku_id"])
def tag(x):
    if x <= 0.2: return "A"
    if x <= 0.5: return "B"
    return "C"
abc["class"] = abc["cumshare"].apply(tag)

# Map back to rows
key_series = feat["store_id"].astype(str) + "||" + feat["sku_id"].astype(str)
abc_key = abc.copy()
abc_key["key"] = abc_key["store_id"].astype(str) + "||" + abc_key["sku_id"].astype(str)
class_map = dict(zip(abc_key["key"], abc_key["class"]))
feat["abc_class"] = key_series.map(class_map)

# Metrics by class
by_class = []
for cls in ["A","B","C"]:
    m = feat["abc_class"]==cls
    if m.any():
        by_class.append({
            "class": cls,
            "wape": WAPE(feat.loc[m,"qty"], preds_oof[m]),
            "smape": SMAPE(feat.loc[m,"qty"], preds_oof[m]),
            "bias": BIAS(feat.loc[m,"qty"], preds_oof[m]),
        })
pd.DataFrame(by_class)


,class,wape,smape,bias
0,A,0.824098,1.600704,-0.788247
1,B,0.822670,1.598367,-0.792289
2,C,0.826055,1.557253,-0.780675


In [14]:
# Save predictions & metrics
out = feat[["store_id","sku_id","date","qty"]].copy()
out["pred_oof"] = preds_oof
out.to_csv(f"{REPORTS}/oof_predictions.csv", index=False)

with open(f"{REPORTS}/cv_metrics.json","w") as f:
    json.dump({"cv_avg":cv_avg, "by_class":by_class}, f, indent=2)

# Train final models on ALL data & save
clf.fit(X, y_occ)
mask_pos_all = y_size > 0
reg.fit(X[mask_pos_all], y_size[mask_pos_all])

import joblib
joblib.dump({"occ_model": clf, "size_model": reg, "features": feature_cols}, f"{MODELS}/two_stage_lgbm.pkl")

print("Saved:", f"{REPORTS}/oof_predictions.csv", f"{REPORTS}/cv_metrics.json", f"{MODELS}/two_stage_lgbm.pkl", sep="\n- ")


[LightGBM] [Info] Number of positive: 61019, number of negative: 4651
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2616
[LightGBM] [Info] Number of data points in the train set: 65670, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.929176 -> initscore=2.574103
[LightGBM] [Info] Start training from score 2.574103
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2616
[LightGBM] [Info] Number of data points in the train set: 61019, number of used features: 16
[LightGBM] [Info] Start training from score 6.977711
Saved:
- reports/oof_predictions.csv
- reports/cv_metrics.json
- models/two_stage_lgbm.pkl


In [15]:
# If you still have `feat` and `preds_oof` in memory, use them.
# Otherwise fall back to reports/oof_predictions.csv (works either way).

import numpy as np, pandas as pd, json, os

if "feat" in globals() and "preds_oof" in globals():
    work = feat[["store_id","sku_id","date","qty"]].copy()
    work["pred_oof"] = preds_oof
else:
    work = pd.read_csv("reports/oof_predictions.csv")
    # make sure types are consistent
    for c in ["store_id","sku_id"]:
        if c in work.columns:
            work[c] = work[c].astype(str)

# ---- Build A/B/C classes by cumulative volume over (store_id, sku_id) ----
pair_vol = (work.groupby(["store_id","sku_id"])["qty"]
                .sum()
                .sort_values(ascending=False))
cumshare = (pair_vol.cumsum() / pair_vol.sum())

def _cls(x):
    if x <= 0.20: return "A"
    if x <= 0.50: return "B"
    return "C"

pair_class = cumshare.to_frame("cumshare").assign(abc_class=cumshare.map(_cls))
pair_class = pair_class["abc_class"]

# Map class back to rows
key = work["store_id"].astype(str) + "||" + work["sku_id"].astype(str)
pair_key = (pair_class.reset_index()
                     .assign(key=lambda d: d["store_id"].astype(str) + "||" + d["sku_id"].astype(str))
                     .set_index("key")["abc_class"])
work["abc_class"] = key.map(pair_key).fillna("C")

# ---- Metrics helpers ----
def WAPE(y, yhat, eps=1e-9):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.abs(y - yhat).sum() / (np.abs(y).sum() + eps))

def SMAPE(y, yhat, eps=1e-9):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.mean(2*np.abs(y - yhat)/(np.abs(y)+np.abs(yhat)+eps)))

def BIAS(y, yhat, eps=1e-9):
    y = np.asarray(y); yhat = np.asarray(yhat)
    return float((yhat - y).sum() / (np.abs(y).sum() + eps))

# ---- Overall + by-class metrics (pre-debias) ----
overall = {
    "wape": WAPE(work["qty"], work["pred_oof"]),
    "smape": SMAPE(work["qty"], work["pred_oof"]),
    "bias": BIAS(work["qty"], work["pred_oof"]),
}
by_class = []
for cls in ["A","B","C"]:
    m = work["abc_class"]==cls
    if m.any():
        by_class.append({
            "class": cls,
            "wape": WAPE(work.loc[m,"qty"], work.loc[m,"pred_oof"]),
            "smape": SMAPE(work.loc[m,"qty"], work.loc[m,"pred_oof"]),
            "bias": BIAS(work.loc[m,"qty"], work.loc[m,"pred_oof"]),
            "rows": int(m.sum())
        })

print("OVERALL (pre-debias):", overall)
pd.DataFrame(by_class)


OVERALL (pre-debias): {'wape': 0.8246967285404175, 'smape': 1.559413472356703, 'bias': -0.785505751702002}


,class,wape,smape,bias,rows
0,A,0.824098,1.600704,-0.788247,1194
1,B,0.822670,1.598367,-0.792289,2189
2,C,0.826055,1.557253,-0.780675,62287


In [16]:
# Bring average bias ~ 0 by scaling predictions so total forecast == total actual.
den = float(work["pred_oof"].sum()) + 1e-9
num = float(work["qty"].sum())
debias_factor = num / den

work["pred_oof_debiased"] = work["pred_oof"] * debias_factor

overall_deb = {
    "wape": WAPE(work["qty"], work["pred_oof_debiased"]),
    "smape": SMAPE(work["qty"], work["pred_oof_debiased"]),
    "bias": BIAS(work["qty"], work["pred_oof_debiased"]),
    "debias_factor": debias_factor
}
by_class_deb = []
for cls in ["A","B","C"]:
    m = work["abc_class"]==cls
    if m.any():
        by_class_deb.append({
            "class": cls,
            "wape": WAPE(work.loc[m,"qty"], work.loc[m,"pred_oof_debiased"]),
            "smape": SMAPE(work.loc[m,"qty"], work.loc[m,"pred_oof_debiased"]),
            "bias": BIAS(work.loc[m,"qty"], work.loc[m,"pred_oof_debiased"]),
            "rows": int(m.sum())
        })

print("OVERALL (post-debias):", overall_deb)
pd.DataFrame(by_class_deb)


OVERALL (post-debias): {'wape': 1.6005504626904081, 'smape': 1.7655955956964817, 'bias': -1.1376944155282864e-16, 'debias_factor': 4.662129674501549}


,class,wape,smape,bias,rows
0,A,1.590590,1.858106,-0.012782,1194
1,B,1.574518,1.857170,-0.031623,2189
2,C,1.618874,1.760604,0.022523,62287


In [17]:
# Write updated OOF with debiased preds
os.makedirs("reports", exist_ok=True)
work_out = work[["store_id","sku_id","date","qty","pred_oof","pred_oof_debiased","abc_class"]].copy()
work_out.to_csv("reports/oof_predictions_debiased.csv", index=False)

# Save a tidy metrics JSON (pre/post)
metrics_payload = {
    "overall_pre": overall,
    "overall_post": overall_deb,
    "by_class_pre": by_class,
    "by_class_post": by_class_deb
}
with open("reports/cv_metrics_fixed.json","w") as f:
    json.dump(metrics_payload, f, indent=2)

# Save factor for use at inference time (multiply future forecasts by this)
with open("models/debias_factor.json","w") as f:
    json.dump({"debias_factor": float(debias_factor)}, f)

print("Saved:")
print("- reports/oof_predictions_debiased.csv")
print("- reports/cv_metrics_fixed.json")
print("- models/debias_factor.json")


Saved:
- reports/oof_predictions_debiased.csv
- reports/cv_metrics_fixed.json
- models/debias_factor.json


In [2]:
# --- OOF-only metrics (rebuild the same validation windows) ---

import os, json
import numpy as np
import pandas as pd

# Load OOF predictions saved earlier by your training cells
work = pd.read_csv("reports/oof_predictions.csv", parse_dates=["date"])

# If these aren't defined above, set them here
HORIZON_DAYS   = 14
MIN_TRAIN_DAYS = 150
N_SPLITS       = 4

def rolling_val_date_windows(dates, n_splits=N_SPLITS, min_train_days=MIN_TRAIN_DAYS, holdout_days=HORIZON_DAYS):
    ud = np.array(sorted(pd.Series(dates).dt.normalize().unique()))
    wins = []
    for i in range(n_splits):
        train_end = len(ud) - (n_splits - i) * holdout_days
        if train_end < min_train_days:
            continue
        val_days = ud[train_end:train_end+holdout_days]
        wins.append(pd.DatetimeIndex(val_days))
    return wins

val_windows = rolling_val_date_windows(work["date"])

# --- FIX: Flatten all DateTimeIndex windows (no union_many in pandas) ---
if len(val_windows) == 0:
    raise ValueError("No validation windows computed. Check dates or CV params.")
val_set = pd.DatetimeIndex(
    np.unique(np.concatenate([w.values for w in val_windows]))
)

# Mark validation rows
work["is_valid"] = work["date"].dt.normalize().isin(val_set)

def WAPE(y, yhat, eps=1e-9):
    y   = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.abs(y - yhat).sum() / (np.abs(y).sum() + eps))

def SMAPE(y, yhat, eps=1e-9):
    y   = np.asarray(y); yhat = np.asarray(yhat)
    return float(np.mean(2*np.abs(y - yhat)/(np.abs(y)+np.abs(yhat)+eps)))

def BIAS(y, yhat, eps=1e-9):
    y   = np.asarray(y); yhat = np.asarray(yhat)
    return float((yhat - y).sum() / (np.abs(y).sum() + eps))

# Overall OOF metrics (validation rows only)
m = work["is_valid"]
overall_oof = {
    "wape": WAPE(work.loc[m, "qty"], work.loc[m, "pred_oof"]),
    "smape": SMAPE(work.loc[m, "qty"], work.loc[m, "pred_oof"]),
    "bias": BIAS(work.loc[m, "qty"], work.loc[m, "pred_oof"]),
    "rows": int(m.sum())
}
overall_oof


{'wape': 0.12252644036363322,
 'smape': 0.4050997645856912,
 'bias': 0.07364243695226826,
 'rows': 13860}

In [3]:
# --- Debias on OOF rows only, then save artifacts ---

# Scale so total forecast == total actual on the OOF slice (bias ~ 0)
num = float(work.loc[m, "qty"].sum())
den = float(work.loc[m, "pred_oof"].sum()) + 1e-9
debias_factor_oof = num / den

work["pred_oof_debiased"] = work["pred_oof"] * debias_factor_oof

overall_oof_post = {
    "wape": WAPE(work.loc[m, "qty"], work.loc[m, "pred_oof_debiased"]),
    "smape": SMAPE(work.loc[m, "qty"], work.loc[m, "pred_oof_debiased"]),
    "bias": BIAS(work.loc[m, "qty"], work.loc[m, "pred_oof_debiased"]),
    "rows": int(m.sum()),
    "debias_factor_oof": debias_factor_oof
}

print("OOF (pre-debias):", overall_oof)
print("OOF (post-debias):", overall_oof_post)

# Save clean files
os.makedirs("reports", exist_ok=True)
os.makedirs("models", exist_ok=True)

work.to_csv("reports/oof_predictions_oofonly.csv", index=False)
with open("reports/cv_metrics_oofonly.json","w") as f:
    json.dump({"overall_oof_pre": overall_oof, "overall_oof_post": overall_oof_post}, f, indent=2)
with open("models/debias_factor_oof.json","w") as f:
    json.dump({"debias_factor_oof": float(debias_factor_oof)}, f)

print("Saved:")
print("- reports/oof_predictions_oofonly.csv")
print("- reports/cv_metrics_oofonly.json")
print("- models/debias_factor_oof.json")


OOF (pre-debias): {'wape': 0.12252644036363322, 'smape': 0.4050997645856912, 'bias': 0.07364243695226826, 'rows': 13860}
OOF (post-debias): {'wape': 0.10246324773588114, 'smape': 0.3945764006208121, 'bias': -1.2633210958279246e-16, 'rows': 13860, 'debias_factor_oof': 0.9314087871178826}
Saved:
- reports/oof_predictions_oofonly.csv
- reports/cv_metrics_oofonly.json
- models/debias_factor_oof.json


In [5]:
# === Forecast next 14 days (no dependency on `df`) ===
# Reads history from reports/oof_predictions.csv and side tables from data/raw,
# loads your saved two-stage models, applies debias_factor_oof, and writes forecasts.

import os, json
import numpy as np
import pandas as pd
import joblib

H = 14
RAW = "data/raw"
REPORTS = "reports"
MODELS = "models"

# 1) Load models + feature list
art = joblib.load(f"{MODELS}/two_stage_lgbm.pkl")
clf, reg, FEATS = art["occ_model"], art["size_model"], art["features"]

# 2) Debias factor (OOF-only); fall back to 1.0 if missing
debias_factor = 1.0
try:
    with open(f"{MODELS}/debias_factor_oof.json") as f:
        debias_factor = float(json.load(f)["debias_factor_oof"])
except Exception:
    print("NOTE: models/debias_factor_oof.json not found; using factor=1.0")

# 3) History (qty per store×sku×day) from OOF file
hist = pd.read_csv(f"{REPORTS}/oof_predictions.csv", parse_dates=["date"])
hist = hist.sort_values(["store_id","sku_id","date"]).reset_index(drop=True)

last_hist_date = hist["date"].max()
future_dates = pd.date_range(last_hist_date + pd.Timedelta(days=1), periods=H, freq="D")

# 4) Side tables for future exogenous values
txn = pd.read_csv(f"{RAW}/transactions.csv", parse_dates=["date"]).rename(columns={"store_nbr":"store_id"})
oil = pd.read_csv(f"{RAW}/oil.csv", parse_dates=["date"]).rename(columns={"dcoilwtico":"oil_price"})
hol = pd.read_csv(f"{RAW}/holidays_events.csv", parse_dates=["date"])

# store median transactions (last 28 days within history window)
txn_recent = txn[txn["date"] <= last_hist_date].sort_values(["store_id","date"])
txn_med = (txn_recent.groupby("store_id")
                     .apply(lambda g: g.tail(28)["transactions"].median())
                     .rename("txn_med"))

# last oil price up to history end
oil = oil.sort_values("date")
oil["oil_price"] = oil["oil_price"].ffill().bfill()
last_oil = float(oil[oil["date"] <= last_hist_date]["oil_price"].iloc[-1])

# simple holiday set (national + additional, not transferred)
hol_flag = (hol[hol["type"].isin(["Holiday","Additional"])]
              .loc[~hol["transferred"].fillna(False), ["date"]]
              .drop_duplicates())
holiday_set = set(hol_flag["date"].dt.normalize().tolist())

# store-level rolling-7 of total qty (last value per store)
store_daily = (hist.groupby(["store_id","date"])["qty"].sum().reset_index())
store_daily = store_daily.sort_values(["store_id","date"])
store_daily["store_roll7"] = (store_daily.groupby("store_id")["qty"]
                              .rolling(7, min_periods=1).mean()
                              .reset_index(level=0, drop=True))
last_store_r7 = (store_daily.groupby("store_id")["store_roll7"]
                 .tail(1).reset_index().set_index("store_id")["store_roll7"])

# forecasting helpers
alpha = 1 - np.exp(-np.log(2)/7.0)  # halflife=7

def forecast_pair(g):
    g = g.sort_values("date").copy()
    store_id = g["store_id"].iloc[0]
    sku_id   = g["sku_id"].iloc[0]

    # last 60 days for lags/rolls
    qty = g["qty"].astype(float).tolist()[-60:]
    ewm = 0.0
    for v in qty:
        ewm = alpha * v + (1 - alpha) * ewm

    store_r7 = float(last_store_r7.get(store_id, np.mean(qty[-7:]) if qty else 0.0))
    tx_med   = float(txn_med.get(store_id, 0.0))

    rows = []
    for d in future_dates:
        def lag(k): return qty[-k] if len(qty) >= k else 0.0
        last7, last28 = (qty[-7:] if len(qty)>=7 else qty[:], qty[-28:] if len(qty)>=28 else qty[:])
        roll7_mean, roll7_std  = (float(np.mean(last7)) if last7 else 0.0, float(np.std(last7)) if last7 else 0.0)
        roll28_mean = float(np.mean(last28)) if last28 else 0.0

        dow, month = d.weekday(), d.month
        is_weekend = int(dow >= 5)
        is_month_end = int((d + pd.Timedelta(days=1)).month != month)

        feat_row = {
            "dow": dow, "month": month, "is_weekend": is_weekend, "is_month_end": is_month_end,
            "lag_1": lag(1), "lag_7": lag(7), "lag_14": lag(14),
            "roll7_mean": roll7_mean, "roll7_std": roll7_std,
            "roll28_mean": roll28_mean, "exp_decay_7": ewm,
            "store_roll7_mean": store_r7,
            "is_promo": 0,  # change if you have promo plans
            "transactions": tx_med,
            "oil_price": last_oil,
            "holiday_flag": 1 if d.normalize() in holiday_set else 0
        }

        x = [feat_row.get(f, 0.0) for f in FEATS]
        p_occ = float(clf.predict_proba([x])[0, 1])
        size  = float(max(0.0, reg.predict([x])[0]))
        yhat  = p_occ * size

        rows.append({"store_id":store_id, "sku_id":sku_id, "date":d,
                     "p_occ":p_occ, "size_pred":size, "forecast_raw":yhat})

        # simulate forward for next day's lags/rolls
        qty.append(yhat)
        ewm = alpha * yhat + (1 - alpha) * ewm

    return pd.DataFrame(rows)

# Run for every pair
pairs = hist[["store_id","sku_id"]].drop_duplicates()
fcasts = []
for _, row in pairs.iterrows():
    g = hist[(hist["store_id"]==row["store_id"]) & (hist["sku_id"]==row["sku_id"])]
    if len(g):
        fcasts.append(forecast_pair(g))

future = pd.concat(fcasts, ignore_index=True) if len(fcasts) else pd.DataFrame(columns=["store_id","sku_id","date","forecast_raw"])
future["forecast"] = future["forecast_raw"] * debias_factor

# Save
os.makedirs(REPORTS, exist_ok=True)
out_path = f"{REPORTS}/forecast_next_14d.csv"
future.to_csv(out_path, index=False)
print(f"Saved forecasts to {out_path}  |  rows: {len(future)}")
future.head()


C:\Users\mnsuh\AppData\Local\Temp\ipykernel_103828\1237077163.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.tail(28)["transactions"].median())


KeyError: "None of ['store_id'] are in the columns"

In [6]:
# === Forecast next 14 days (fixed) ===
# Reads history from reports/oof_predictions.csv & raw side tables,
# loads saved two-stage models, applies OOF debias factor, and writes forecasts.

import os, json
import numpy as np
import pandas as pd
import joblib

H = 14
RAW = "data/raw"
REPORTS = "reports"
MODELS = "models"

# 1) Load models + feature list
art = joblib.load(f"{MODELS}/two_stage_lgbm.pkl")
clf, reg, FEATS = art["occ_model"], art["size_model"], art["features"]

# 2) Debias factor (OOF-only); fall back to 1.0 if missing
debias_factor = 1.0
try:
    with open(f"{MODELS}/debias_factor_oof.json") as f:
        debias_factor = float(json.load(f)["debias_factor_oof"])
except Exception:
    print("NOTE: models/debias_factor_oof.json not found; using factor=1.0")

# 3) History (qty per store×sku×day)
hist = pd.read_csv(f"{REPORTS}/oof_predictions.csv", parse_dates=["date"])
hist = hist.sort_values(["store_id","sku_id","date"]).reset_index(drop=True)

last_hist_date = hist["date"].max()
future_dates = pd.date_range(last_hist_date + pd.Timedelta(days=1), periods=H, freq="D")

# 4) Side tables for future exogenous values
txn = pd.read_csv(f"{RAW}/transactions.csv", parse_dates=["date"]).rename(columns={"store_nbr":"store_id"})
oil = pd.read_csv(f"{RAW}/oil.csv", parse_dates=["date"]).rename(columns={"dcoilwtico":"oil_price"})
hol = pd.read_csv(f"{RAW}/holidays_events.csv", parse_dates=["date"])

# -- FIX 1: transactions median (no deprecation; last 28 days per store)
txn_recent = txn.loc[txn["date"] <= last_hist_date].sort_values(["store_id","date"]).copy()
txn_med = (
    txn_recent.groupby("store_id", group_keys=False)["transactions"]
    .apply(lambda s: s.tail(28).median())
    .rename("txn_med")
)

# Oil price: carry forward the last known value
oil = oil.sort_values("date")
oil["oil_price"] = oil["oil_price"].ffill().bfill()
last_oil = float(oil[oil["date"] <= last_hist_date]["oil_price"].iloc[-1])

# Simple holiday set (national + additional, not transferred)
hol_flag = (
    hol[hol["type"].isin(["Holiday","Additional"])]
    .loc[~hol["transferred"].fillna(False), ["date"]]
    .drop_duplicates()
)
holiday_set = set(hol_flag["date"].dt.normalize().tolist())

# Store-level rolling-7 of total qty (last per store)
store_daily = (
    hist.groupby(["store_id","date"], as_index=False)["qty"].sum()
    .sort_values(["store_id","date"])
)
store_daily["store_roll7"] = (
    store_daily.groupby("store_id")["qty"]
    .rolling(7, min_periods=1).mean()
    .reset_index(level=0, drop=True)
)
# -- FIX 2: get last value per store safely (no missing 'store_id' column)
last_store_r7 = (
    store_daily.groupby("store_id", as_index=True)["store_roll7"]
    .last()
).fillna(0.0)

# forecasting helpers
alpha = 1 - np.exp(-np.log(2)/7.0)  # halflife=7

def forecast_pair(g):
    g = g.sort_values("date").copy()
    store_id = g["store_id"].iloc[0]
    sku_id   = g["sku_id"].iloc[0]

    # last 60 days for lags/rolls
    qty = g["qty"].astype(float).tolist()[-60:]
    ewm = 0.0
    for v in qty:
        ewm = alpha * v + (1 - alpha) * ewm

    store_r7 = float(last_store_r7.get(store_id, np.mean(qty[-7:]) if qty else 0.0))
    tx_med   = float(txn_med.get(store_id, 0.0))

    rows = []
    for d in future_dates:
        def lag(k): return qty[-k] if len(qty) >= k else 0.0
        last7  = qty[-7:]  if len(qty) >= 7  else qty[:]
        last28 = qty[-28:] if len(qty) >= 28 else qty[:]
        roll7_mean  = float(np.mean(last7))  if last7 else 0.0
        roll7_std   = float(np.std(last7))   if last7 else 0.0
        roll28_mean = float(np.mean(last28)) if last28 else 0.0

        dow, month = d.weekday(), d.month
        is_weekend   = int(dow >= 5)
        is_month_end = int((d + pd.Timedelta(days=1)).month != month)

        feat_row = {
            "dow": dow, "month": month, "is_weekend": is_weekend, "is_month_end": is_month_end,
            "lag_1": lag(1), "lag_7": lag(7), "lag_14": lag(14),
            "roll7_mean": roll7_mean, "roll7_std": roll7_std,
            "roll28_mean": roll28_mean, "exp_decay_7": ewm,
            "store_roll7_mean": store_r7,
            "is_promo": 0,                      # adjust if you have promo plans
            "transactions": tx_med,
            "oil_price": last_oil,
            "holiday_flag": 1 if d.normalize() in holiday_set else 0
        }

        x = [feat_row.get(f, 0.0) for f in FEATS]
        p_occ = float(clf.predict_proba([x])[0, 1])
        size  = float(max(0.0, reg.predict([x])[0]))
        yhat  = p_occ * size

        rows.append({
            "store_id": store_id, "sku_id": sku_id, "date": d,
            "p_occ": p_occ, "size_pred": size, "forecast_raw": yhat
        })

        # simulate forward for next day's lags/rolls
        qty.append(yhat)
        ewm = alpha * yhat + (1 - alpha) * ewm

    return pd.DataFrame(rows)

# Run for every pair
pairs = hist[["store_id","sku_id"]].drop_duplicates()
fcasts = []
for _, row in pairs.iterrows():
    g = hist[(hist["store_id"]==row["store_id"]) & (hist["sku_id"]==row["sku_id"])]
    if len(g):
        fcasts.append(forecast_pair(g))

future = pd.concat(fcasts, ignore_index=True) if len(fcasts) else pd.DataFrame(columns=["store_id","sku_id","date","forecast_raw"])
future["forecast"] = future["forecast_raw"] * debias_factor

# Save
os.makedirs(REPORTS, exist_ok=True)
out_path = f"{REPORTS}/forecast_next_14d.csv"
future.to_csv(out_path, index=False)
print(f"Saved forecasts to {out_path}  |  rows: {len(future)}")
future.head()


Saved forecasts to reports/forecast_next_14d.csv  |  rows: 4620


,store_id,sku_id,date,p_occ,size_pred,forecast_raw,forecast
0,3,AUTOMOTIVE,2017-08-16,0.999990,10.243966,10.243865,9.541226
1,3,AUTOMOTIVE,2017-08-17,0.999993,9.166079,9.166018,8.537310
2,3,AUTOMOTIVE,2017-08-18,0.999996,11.224546,11.224503,10.454600
3,3,AUTOMOTIVE,2017-08-19,0.999997,14.316113,14.316075,13.334118
4,3,AUTOMOTIVE,2017-08-20,0.999999,17.286934,17.286917,16.101187


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

oof = pd.read_csv("reports/oof_predictions_oofonly.csv", parse_dates=["date"])
fut = pd.read_csv("reports/forecast_next_14d.csv", parse_dates=["date"])

print("Next 14d total forecast:", fut["forecast"].sum())
last14 = oof.sort_values("date").groupby(["store_id","sku_id"]).tail(14)
print("Last 14d total actual  :", last14["qty"].sum())

by_store = fut.groupby("store_id")["forecast"].sum().sort_values(ascending=False).head(10)
print("\nTop stores by 14d forecast:\n", by_store)

def plot_pair(store_id, sku_id):
    hist = oof[(oof.store_id==store_id) & (oof.sku_id==sku_id)].sort_values("date").tail(60)
    nxt  = fut[(fut.store_id==store_id) & (fut.sku_id==sku_id)].sort_values("date")
    plt.figure(figsize=(9,3))
    plt.plot(hist["date"], hist["qty"], label="actual (last 60d)")
    plt.plot(hist["date"], hist.get("pred_oof_debiased", hist.get("pred_oof")), label="pred (OOF)", alpha=0.7)
    plt.plot(nxt["date"], nxt["forecast"], label="forecast (next 14d)")
    plt.legend(); plt.title(f"Store {store_id} | SKU {sku_id}"); plt.show()

# example (replace with a real pair from your data)
# plot_pair(store_id=1, sku_id="BEVERAGES")


Next 14d total forecast: 3729726.1642138986
Last 14d total actual  : 4162290.0694049997

Top stores by 14d forecast:
 store_id
44    538534.501547
45    491445.931412
47    449342.982488
3     419872.673262
49    410605.604819
46    343242.301690
48    289798.655967
51    286002.146433
50    253849.431103
8     247031.935493
Name: forecast, dtype: float64


In [8]:
import numpy as np, pandas as pd

# Use debiased OOF if available; else use pred_oof
oof = pd.read_csv("reports/oof_predictions_oofonly.csv", parse_dates=["date"])
pred_col = "pred_oof_debiased" if "pred_oof_debiased" in oof.columns else "pred_oof"
oof = oof[oof["is_valid"]] if "is_valid" in oof.columns else oof  # ensure only val rows

oof["resid"] = oof["qty"] - oof[pred_col]

# per-pair residual std; guard for tiny samples
sigma_pair = (oof.groupby(["store_id","sku_id"])["resid"]
                .agg(lambda s: float(np.std(s)) if len(s)>=7 else np.nan)
                .rename("sigma"))

# fallbacks: by store, then global
sigma_store = (oof.groupby("store_id")["resid"].std().rename("sigma_store"))
sigma_global = float(oof["resid"].std())

sig = sigma_pair.reset_index()
sig = sig.merge(sigma_store.reset_index(), on="store_id", how="left")
sig["sigma_filled"] = sig["sigma"].fillna(sig["sigma_store"]).fillna(sigma_global)
sig = sig[["store_id","sku_id","sigma_filled"]].rename(columns={"sigma_filled":"sigma"})
sig.head()


,store_id,sku_id,sigma
0,3,AUTOMOTIVE,4.472280
1,3,BABY CARE,1.601356
2,3,BEAUTY,4.812473
3,3,BEVERAGES,1062.271649
4,3,BOOKS,0.007447


In [9]:
import numpy as np, pandas as pd

# inputs you can change
LEAD_TIME_DAYS = 2
service_z = {"A": 1.645, "B": 1.282, "C": 0.842}  # 95/90/80%

# ABC classes (rebuild quickly from OOF totals)
oof = pd.read_csv("reports/oof_predictions_oofonly.csv", parse_dates=["date"])
pair_vol = oof.groupby(["store_id","sku_id"])["qty"].sum().sort_values(ascending=False)
cum = (pair_vol.cumsum()/pair_vol.sum())
def _cls(x): return "A" if x<=0.2 else ("B" if x<=0.5 else "C")
abc = cum.map(_cls).rename("abc_class").reset_index()

# future sum over lead time
fut = pd.read_csv("reports/forecast_next_14d.csv", parse_dates=["date"])
need = (fut.sort_values("date")
          .groupby(["store_id","sku_id"])["forecast"]
          .apply(lambda s: float(s.head(LEAD_TIME_DAYS).sum()))
          .rename("need_L")
          .reset_index())

# join sigma and ABC
plan = (need.merge(sig, on=["store_id","sku_id"], how="left")
             .merge(abc, on=["store_id","sku_id"], how="left"))

# safety stock = z * sigma * sqrt(L)  (absolute units)
plan["z"] = plan["abc_class"].map(service_z).fillna(service_z["C"])
plan["sigma"] = plan["sigma"].fillna(plan["sigma"].median())  # last resort fallback
plan["safety_stock"] = plan["z"] * plan["sigma"] * np.sqrt(LEAD_TIME_DAYS)

# on-hand/on-order (Favorita has none → 0)
plan["on_hand"] = 0.0
plan["on_order"] = 0.0

# final order qty
plan["order_qty"] = np.maximum(0.0, plan["need_L"] + plan["safety_stock"] - plan["on_hand"] - plan["on_order"])

# tidy + save
cols = ["store_id","sku_id","abc_class","need_L","safety_stock","order_qty"]
order_plan = plan[cols].sort_values(["store_id","order_qty"], ascending=[True, False])
order_plan.to_csv("reports/order_plan_L{}d.csv".format(LEAD_TIME_DAYS), index=False)
print("Saved:", f"reports/order_plan_L{LEAD_TIME_DAYS}d.csv")
order_plan.head(10)


Saved: reports/order_plan_L2d.csv


,store_id,sku_id,abc_class,need_L,safety_stock,order_qty
3,3,BEVERAGES,B,12912.109300,1925.921644,14838.030944
30,3,PRODUCE,B,13577.043730,1168.525354,14745.569084
12,3,GROCERY I,B,12561.338603,1257.841116,13819.179719
8,3,DAIRY,C,4188.820095,270.295587,4459.115681
7,3,CLEANING,C,3549.418772,382.100357,3931.519128
5,3,BREAD/BAKERY,C,2245.404625,146.834981,2392.239606
28,3,POULTRY,C,2157.491713,146.520202,2304.011915
24,3,MEATS,C,1111.722762,112.154403,1223.877165
25,3,PERSONAL CARE,C,988.152608,101.758965,1089.911573
18,3,HOME CARE,C,957.799902,118.480678,1076.280580
